<a href="https://colab.research.google.com/github/ajibigad/ML-Playground/blob/main/breast_cancer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

from google.colab import files, drive
import shutil

if not os.path.ismount('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

destination = "/content/drive/MyDrive/Colab Notebooks/breast-cancer.csv"  # Set the destination path in your Drive

if not os.path.exists(destination):
  uploaded = files.upload()
  filename = list(uploaded.keys())[0]  # Get the uploaded filename
  shutil.move(filename, destination)
  print(f"Uploaded file '{filename}' has been moved to Google Drive at '{destination}'")
else:
  print(f"{destination} already exists")

Google Drive is already mounted.
/content/drive/MyDrive/Colab Notebooks/breast-cancer.csv already exists


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load CSV file into a pandas DataFrame
df = pd.read_csv(destination)

# Convert M -> 1 and B -> 0
label_encoder = LabelEncoder()
df['diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

X = df.drop(['id', 'diagnosis'], axis=1).values
y = df['diagnosis'].values

In [5]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape targets to rank 2 tensor eg. from (455) to (455, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape targets to (455, 1)

# Z-score normalization
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)

X_train_normalized = (X_train_tensor - mean) / std
X_test_normalized = (X_test_tensor - mean) / std

In [6]:
import pdb
import numpy as np

def sigmoid(x): return 1/(1+torch.exp(-x))

def abs_mean_loss(predictions, targets):
  # // abs mean or mse but we did abs mean loss here
  # // for every prediction, if the target is 1, then difference is 1-pred, else difference is abs(0-pred)
  # // predictions will always be between 0 and 1 due to sigmond function
  return torch.where(targets==1, 1-predictions, predictions).mean()

def mse(preds, targets): return ((preds-targets)**2).mean()

def batch_accuracy(predictions, targets):
  with torch.no_grad():
    correct = (predictions>=0.5) == targets
    return correct.float().mean()

def validate_epoch(model, val_data, val_target):
    accs = [batch_accuracy(model.predict(xb), yb) for xb,yb in zip(val_data, val_target)]
    return round(torch.stack(accs).mean().item(), 4)

class LinearModel():
  def __init__(self, weight, bias, x, y, x_validate, y_validate, loss_fn, metrics):
    self.weight = weight
    self.bias = bias
    self.x = x
    self.y = y
    self.x_validate = x_validate
    self.y_validate = y_validate
    self.params = self.weight, self.bias
    self.loss_fn = loss_fn
    self.metrics = metrics
    self.z = None

  def predict(self, x):
    # z(x) = xw + b
    # y = g(z(x)) where g is an activation function
    self.z = x@self.weight + self.bias

    if torch.isnan(self.z).any():
      pdb.set_trace()

    # apply activation function
    return sigmoid(self.z)

  def step(self, loss, learning_rate):
    # get loss
    # calculate gradient, loss.backward()
    # update parameters using learning rate
    loss.backward()
    for p in self.params:
      p.data -= p.grad.data * learning_rate
      p.grad.zero_()

  def train(self, epoch, learning_rate):
    for i in range(epoch):
      preds = self.predict(self.x)
      try:
        loss = self.loss_fn(preds, self.y)
      except:
        print(preds.view(-1).tolist(), self.z)
        raise
      accuracy = self.metrics(preds, self.y)
      validation = validate_epoch(self, self.x_validate, self.y_validate)
      print(f"epoch {i}: loss: {loss}, accuracy: {accuracy}, validation: {validation}")
      self.step(loss, learning_rate)

def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [7]:
w = init_params((X_train_normalized.shape[1], 1)) # size number of
b = init_params(1)

model = LinearModel(w, b, X_train_normalized, y_train_tensor, X_test_normalized, y_test_tensor, nn.BCELoss(), batch_accuracy)
model.train(epoch=50, learning_rate=1e-1)

epoch 0: loss: 2.5924503803253174, accuracy: 0.32087913155555725, validation: 0.3158
epoch 1: loss: 2.200493097305298, accuracy: 0.38461539149284363, validation: 0.3684
epoch 2: loss: 1.8839497566223145, accuracy: 0.4263736307621002, validation: 0.4035
epoch 3: loss: 1.6269292831420898, accuracy: 0.4703296720981598, validation: 0.4825
epoch 4: loss: 1.4190452098846436, accuracy: 0.5010989308357239, validation: 0.5
epoch 5: loss: 1.2500863075256348, accuracy: 0.5428571701049805, validation: 0.5351
epoch 6: loss: 1.111772060394287, accuracy: 0.5692307949066162, validation: 0.5702
epoch 7: loss: 0.997462272644043, accuracy: 0.6087912321090698, validation: 0.5965
epoch 8: loss: 0.9020546078681946, accuracy: 0.6417582631111145, validation: 0.6491
epoch 9: loss: 0.8217980265617371, accuracy: 0.6681318879127502, validation: 0.6754
epoch 10: loss: 0.7538639307022095, accuracy: 0.6813187003135681, validation: 0.6842
epoch 11: loss: 0.69603031873703, accuracy: 0.7120879292488098, validation: 0.7

In [8]:
def random_test(model):
  test_X = X[35:67]
  test_y = y[35:67]

  test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
  test_X_tensor_normalized = (test_X_tensor - mean) / std

  # print(test_X)

  preds = (model.predict(test_X_tensor_normalized).view(-1) >= 0.5).tolist()
  targets = test_y >= 0.5
  for pred, target in zip(preds, targets):
    print(f"pred: {pred:>1}, target {target:>1},   match: {pred==target:>1}")

In [9]:
random_test(model)
model.params

pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 1,   match: 0
pred: 0, target 1,   match: 0
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1


(tensor([[ 0.5258],
         [ 0.8653],
         [ 0.1862],
         [ 0.3394],
         [ 1.4118],
         [ 0.6473],
         [-0.1412],
         [ 0.5237],
         [-1.2163],
         [ 0.2542],
         [-0.5325],
         [ 0.9348],
         [ 2.0133],
         [-0.1452],
         [ 0.7536],
         [-1.1988],
         [-0.8155],
         [ 0.1698],
         [ 0.7537],
         [-1.3546],
         [ 0.0174],
         [-1.3198],
         [ 3.0065],
         [ 0.3005],
         [-0.5575],
         [ 0.8816],
         [-0.5776],
         [ 1.1805],
         [ 0.1884],
         [ 1.0792]], requires_grad=True),
 tensor([0.2826], requires_grad=True))

In [10]:
class NeuralNet(LinearModel):
  def __init__(self, layers, x, y, x_validate, y_validate, loss_fn, metrics):
    self.layers = layers
    self.x = x
    self.y = y
    self.x_validate = x_validate
    self.y_validate = y_validate
    self.loss_fn = loss_fn
    self.metrics = metrics
    self.z = None

  def predict(self, x):
    for layer in self.layers[:-1]:
      res = x@layer[0] + layer[1]
      res = res.max(torch.tensor(0.0))
      if torch.isnan(res).any():
        print("nan after during prediction in inner layers")
        # pdb.set_trace()

    res = res@self.layers[-1][0] + self.layers[-1][1]
    self.z = res
    if torch.isnan(self.z).any():
      print("nan after during prediction")
      # pdb.set_trace()
    return sigmoid(res)

  def step(self, loss, learning_rate):
    loss.backward()
    for layer in self.layers:
      for p in layer:
        p.data -= p.grad.data * learning_rate
        if torch.isnan(p).any():
          print("nan after backpropagation")
          # pdb.set_trace()
        p.grad.zero_()


In [11]:
neurons = 3
w1 = init_params((30,neurons))
b1 = init_params(neurons)
w2 = init_params((neurons,1))
b2 = init_params(1)

layer1 = [w1, b1]
layer2 = [w2, b2]
layers = [layer1, layer2]

model2 = NeuralNet(layers, X_train_normalized, y_train_tensor, X_test_normalized, y_test_tensor, nn.BCELoss(), batch_accuracy)
model2.train(epoch=50, learning_rate=1e-1)

epoch 0: loss: 1.4934221506118774, accuracy: 0.4703296720981598, validation: 0.5351
epoch 1: loss: 0.8311249613761902, accuracy: 0.6175824403762817, validation: 0.6842
epoch 2: loss: 0.5113006830215454, accuracy: 0.7626373767852783, validation: 0.8246
epoch 3: loss: 0.38411933183670044, accuracy: 0.8461538553237915, validation: 0.8684
epoch 4: loss: 0.3271823823451996, accuracy: 0.8813186883926392, validation: 0.9035
epoch 5: loss: 0.2972872853279114, accuracy: 0.8857142925262451, validation: 0.9123
epoch 6: loss: 0.2790103852748871, accuracy: 0.892307698726654, validation: 0.9386
epoch 7: loss: 0.26611244678497314, accuracy: 0.898901104927063, validation: 0.9386
epoch 8: loss: 0.2556736171245575, accuracy: 0.903296709060669, validation: 0.9386
epoch 9: loss: 0.24549831449985504, accuracy: 0.903296709060669, validation: 0.9386
epoch 10: loss: 0.23618640005588531, accuracy: 0.898901104927063, validation: 0.9474
epoch 11: loss: 0.22777873277664185, accuracy: 0.9076923131942749, validatio

In [12]:
random_test(model2)
model2.layers

pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 1,   match: 0
pred: 1, target 1,   match: 1
pred: 0, target 1,   match: 0
pred: 0, target 1,   match: 0
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1


[[tensor([[ 1.2173,  0.1943, -0.2988],
          [-0.0109,  0.9105,  0.4443],
          [-0.1609, -0.8381, -0.8514],
          [-1.0652,  0.0607, -0.1899],
          [ 0.1911, -2.3014, -0.2359],
          [-0.3281, -0.9118, -0.4874],
          [-1.3552,  0.0598,  0.6202],
          [-1.3204,  0.9788,  1.3647],
          [ 0.3760, -0.6247,  0.2586],
          [-0.2069, -0.0730, -0.2081],
          [ 0.0352,  0.7597, -0.7034],
          [ 0.2611, -2.4341,  0.0490],
          [-0.8986, -0.0538, -0.4731],
          [ 0.0374,  1.2350, -0.0407],
          [-2.2704,  0.2302,  1.1263],
          [-1.9125, -0.5432, -0.4881],
          [ 1.7176,  0.4430,  0.3617],
          [-0.1561,  0.4991, -1.0878],
          [-0.6286,  0.9054,  0.3999],
          [ 0.5534, -0.9575, -0.2215],
          [-0.0257, -0.0269,  2.5339],
          [ 0.8283, -0.6421, -0.1836],
          [ 1.4056, -1.7520,  0.9900],
          [ 0.2671, -1.8218,  1.7101],
          [ 0.2835, -0.0185, -0.6755],
          [-0.3127,  2.01